# North Carolina Scorecard

This notebook generates the county, school district, and statewide "future voter scorecards" for NC. It is generalize to be updated every month, with minimal changes.

Scorecard outputs (tables) are written back to BigQuery, where they are then read into Google Sheets for formatting

In [42]:
import pandas as pd
import numpy as np
import pandas_gbq

## Inputs
Update the fields below each month

In [43]:
# Inputs
as_of_data_date = pd.Timestamp("2025-10-18")
acs_year = '2022' # 2022 for 2024 scorecards. ACS vintages trail by 2 years


## Outputs
Run the cells below, without edits each month

In [44]:
# Derived variables
latest_18_yob = as_of_data_date.year - 18 # 2006 for 2024 scorecards
earliest_18_yob = latest_18_yob - 1 # 2005 for 2024 scorecards

latest_45_yob = as_of_data_date.year - 45 # 1979 for 2024 scorecards
earliest_45_yob = latest_45_yob - 1 # 1978 for 2024 scorecards

data_date_suffix = str(as_of_data_date.year) + str(as_of_data_date.month).rjust(2, "0") + str(as_of_data_date.day).rjust(2, "0")  # this is the rolling "as of" date, where we snap the line for 18yos

# Define columns

    # Voter file
REG_YOB_LATE_YEAR = 'REG_YOB_' + str(latest_18_yob) # col name for number of registrants with given birth year (2006 for 2024 scorecards)
REG_YOB_EARLY_YEAR = 'REG_YOB_' + str(earliest_18_yob) # col name for number of registrants with given birth year (2005 for 2024 scorecards)

REG_45_PLUS_YOB_LATE_YEAR = 'REG_45_PLUS_YOB_' + str(latest_45_yob) # col name for number of registrants with given birth year (1979 for 2024 scorecards)
REG_45_PLUS_YOB_EARLY_YEAR = 'REG_45_PLUS_YOB_' + str(earliest_45_yob) # col name for number of registrants with given birth year (1978 for 2024 scorecards)

    # ACS
EST_18_YO_THIS_YEAR = 'EST_18_YO_' + str(as_of_data_date.year) # col name for number of 18 yos this "as_of" year estimated from ACS
EST_18_AND_19_YO_THIS_YEAR = 'EST_18_AND_19_YO_' + str(as_of_data_date.year) # col name for number of 18 and 19 yos this "as_of" year estimated from ACS

EST_45_PLUS_YO_THIS_YEAR = 'EST_45_PLUS_YO_' + str(as_of_data_date.year) # col name for number of 45+ yos in the "as of" year estimated from ACS


### County Scorecard

In [45]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
voter_source_table = data_date_suffix + "_nc_voter_registration_and_history"

acs_S0101_table = "S0101_us_counties_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file by county, counting the number of registrants in a given birth year.
* Then, left joins the county estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [46]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = '95' AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = '68' AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_county AS(
    SELECT
    COUNTY_FIPS,
    COUNTY_NAME,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY COUNTY_FIPS, COUNTY_NAME

), acs_county AS(
    SELECT
    COUNTY_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_county.*,
acs_county.EST_15_TO_17_YO,
acs_county.MOE_15_TO_17_YO,
acs_county.""" + EST_18_YO_THIS_YEAR + """,
acs_county.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_county.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_county LEFT JOIN acs_county ON voter_file_county.COUNTY_FIPS = acs_county.COUNTY_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [47]:
# Preview
df.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,067,FORSYTH,230316,2565,3715,136219,132960,15559,27,5186.333333,10372.666667,186737
1,147,PITT,104023,1541,2616,55861,54351,6294,28,2098.000000,4196.000000,71236
2,063,DURHAM,198645,1795,2604,100808,97823,10224,39,3408.000000,6816.000000,139455
3,189,WATAUGA,34378,294,481,17706,17276,1202,29,400.666667,801.333333,23535
4,033,CASWELL,13497,77,159,9225,9064,783,46,261.000000,522.000000,13432


In [48]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [49]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.20054945054945056


In [50]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [51]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20251018,EST_REG_RATE_18_YO_AS_OF_20251018
6,183,WAKE,742674,8864,12769,401529,388603,47558,30,15852.666667,31705.333333,499266,11424.815934,0.720687
84,119,MECKLENBURG,688387,7411,10880,348176,336957,42911,18,14303.666667,28607.333333,465789,9592.978022,0.670666
91,081,GUILFORD,326953,3379,5478,190495,185600,21093,*****,7031.000000,14062.000000,253633,4477.60989,0.636838
0,067,FORSYTH,230316,2565,3715,136219,132960,15559,27,5186.333333,10372.666667,186737,3310.041209,0.638224
2,063,DURHAM,198645,1795,2604,100808,97823,10224,39,3408.000000,6816.000000,139455,2317.230769,0.679939


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [52]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.7972602739726027


In [53]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [54]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head(20)

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20251018,EST_REG_RATE_18_YO_AS_OF_20251018,EST_REG_45_PLUS_YO_AS_OF_20251018,EST_REG_RATE_45_PLUS_YO_AS_OF_20251018
6,183,WAKE,742674,8864,12769,401529,388603,47558,30,15852.666667,31705.333333,499266,11424.815934,0.720687,398908.386301,0.79899
84,119,MECKLENBURG,688387,7411,10880,348176,336957,42911,18,14303.666667,28607.333333,465789,9592.978022,0.670666,345901.463014,0.742614
91,081,GUILFORD,326953,3379,5478,190495,185600,21093,*****,7031.000000,14062.000000,253633,4477.60989,0.636838,189502.589041,0.747153
0,067,FORSYTH,230316,2565,3715,136219,132960,15559,27,5186.333333,10372.666667,186737,3310.041209,0.638224,135558.271233,0.725932
2,063,DURHAM,198645,1795,2604,100808,97823,10224,39,3408.000000,6816.000000,139455,2317.230769,0.679939,100202.821918,0.718532
19,021,BUNCOMBE,179689,1527,2128,109918,107002,8663,38,2887.666667,5775.333333,144295,1953.769231,0.676591,109326.810959,0.757662
29,051,CUMBERLAND,174118,2132,3093,97384,94693,12860,36,4286.666667,8573.333333,129853,2752.299451,0.642061,96838.427397,0.745754
58,179,UNION,158097,2556,3410,94513,91587,12970,5,4323.333333,8646.666667,115787,3239.873626,0.749393,93919.783562,0.811143
78,129,NEW HANOVER,156894,1348,2148,93766,91524,7279,81,2426.333333,4852.666667,115498,1778.78022,0.733115,93311.457534,0.807905
25,101,JOHNSTON,144704,1809,2691,82956,80480,9984,98,3328.000000,6656.000000,103011,2348.678571,0.705733,82454.016438,0.800439


In [55]:
df_reg_est = df_reg_est.sort_values('EST_REG_RATE_18_YO_AS_OF_20251018', ascending=False)
df_reg_est

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20251018,EST_REG_RATE_18_YO_AS_OF_20251018,EST_REG_45_PLUS_YO_AS_OF_20251018,EST_REG_RATE_45_PLUS_YO_AS_OF_20251018
99,177,TYRRELL,1878,10,21,1342,1313,36,39,12.000000,24.000000,1839,14.211538,1.184295,1336.120548,0.726547
1,147,PITT,104023,1541,2616,55861,54351,6294,28,2098.000000,4196.000000,71236,2065.637363,0.984575,55554.863014,0.779871
3,189,WATAUGA,34378,294,481,17706,17276,1202,29,400.666667,801.333333,23535,390.464286,0.974536,17618.821918,0.748622
39,141,PENDER,45192,556,726,28406,27617,2379,130,793.000000,1586.000000,32899,701.598901,0.88474,28246.038356,0.858568
55,019,BRUNSWICK,125779,797,1121,95839,94652,3532,58,1177.333333,2354.666667,97646,1021.815934,0.867907,95598.347945,0.97903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,145,PERSON,24419,163,343,16190,15850,1476,87,492.000000,984.000000,21962,231.788462,0.471115,16121.068493,0.734044
14,173,SWAIN,8274,61,112,5475,5339,584,98,194.666667,389.333333,7158,83.461538,0.428741,5447.427397,0.761026
4,033,CASWELL,13497,77,159,9225,9064,783,46,261.000000,522.000000,13432,108.887363,0.417193,9192.358904,0.684363
63,103,JONES,6131,36,78,4172,4072,392,71,130.666667,261.333333,5813,51.642857,0.395226,4151.726027,0.714214


#### Output
Write back to BQ

##### Wide

In [56]:
# Flag largest counties
df_reg_est['is_in_10_largest'] = np.where(df_reg_est.COUNTY_NAME.isin(df_reg_est.nlargest(10, columns='EST_18_YO_2025').COUNTY_NAME),1,0)

In [57]:
#flag small counties
df_reg_est['not_sufficient_data'] = np.where(df_reg_est['EST_18_YO_2025'] > 100, 1, 0)


In [58]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_county_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


### Statewide Scorecard

In [59]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
acs_S0101_table = "S0101_us_states_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file for NC state, counting the number of registrants in a given birth year.
* Then, left joins the statewide estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos is derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [60]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = '95' AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = '68' AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_nc AS(
    SELECT
    STATE_FIPS,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY STATE_FIPS

), acs_nc AS(
    SELECT
    STATE_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_nc.*,
acs_nc.EST_15_TO_17_YO,
acs_nc.MOE_15_TO_17_YO,
acs_nc.""" + EST_18_YO_THIS_YEAR + """,
acs_nc.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_nc.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_nc LEFT JOIN acs_nc ON voter_file_nc.STATE_FIPS = acs_nc.STATE_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [61]:
# Preview
df

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,37,6546636,70341,104974,3905759,3808620,404849,794,134949.666667,269899.333333,5139416


In [62]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [63]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.20054945054945056


In [64]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [65]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20251018,EST_REG_RATE_18_YO_AS_OF_20251018
0,37,6546636,70341,104974,3905759,3808620,404849,794,134949.666667,269899.333333,5139416,91393.478022,0.677241


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [66]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.7972602739726027


In [67]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [68]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20251018,EST_REG_RATE_18_YO_AS_OF_20251018,EST_REG_45_PLUS_YO_AS_OF_20251018,EST_REG_RATE_45_PLUS_YO_AS_OF_20251018
0,37,6546636,70341,104974,3905759,3808620,404849,794,134949.666667,269899.333333,5139416,91393.478022,0.677241,3886065.065753,0.75613


#### Output
Write back to BQ

In [69]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_statewide_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
